### DNN For Elastic Property Prediction

#### 1. Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt

#### 2. Initialize Parameters

In [2]:
# function for initializing parameters for L-layer neural network

def initialize_parameters(dim_layers):
    """ 
    The arguments are,
    dim_layers ---> A python array containing the dimensions of each layers in the network
    
    Function returs are,
    parameters ---> A python dictionary containing the parameters of the network. i.e., Weights and bias in each layer
                    W1,b1,W2,b2,.......WL,bL
                    Where W1 : Weight matrix of shape (dim_layers[l], dim_layers[l-1])
                    b1 : Bias vector of shape (dim_layers[l], 1)
    """
    parameters = {}
    L = len(dim_layers)   # total number of layers in the neural network
    
    for l in range(1, L):
        parameters["W" + str(l)] = np.random.randn(dim_layers[l], dim_layers[l-1]) * 0.01
        parameters["b" + str(l)] = np.zeros((dim_layers[l], 1))
        
    return parameters

#### 3. Forward propagation

#### 3.1 Linear forward function

In [3]:
# function for linear forward

def linear_forward(A, W, b):
    """
    Arguments are,
    A ---> Activations from the previous layer / input data
    W ---> Weight matrix of shape (dim_layers[l], dim_layers[l-1])
    b ---> Bias vector of shape (dim_layers[l], 1)
    
    Function returns are,
    Z ---> Input for the activation function (Pre-activation parameter)
    backup_values ---> for storing the values of A, W and b for Backward propagation - tuple
    
    """
    
    Z = np.dot(W,A) + b
    backup_values = (A, W, b)
    
    return Z, backup_values

#### 3.2 Linear forward activation function

In [5]:
# function for linear forward activation

def forward_activation(A_prev_layer, W, b, activation = "relu"):
    """
    Arguments are,
    A_prev_layer ---> Activation from the previous layer / input data
    W --- > Weight matrix of shape (dim_layers[l], dim_layers[l-1])
    b ---> Bias vector of shape (dim_layers[l], 1)
    activation ---> Activation to be used (ReLu) i.e., A = RELU(Z) = max(0,Z)
    
    Function returns are,
    A ---> Output of activation function (Post-activation parameter)
    backup_values ---> for storing the values from linear forward calculation and activation function calculations
    
    """
    
    Z, linear_backup_values = linear_forward(A_prev_layer, W, b)
    A, activation_backup_values = np.max(0, Z)
    
    backup_values = (linear_backup_values, activation_backup_values)
    
    return A, backup_values

#### 3.3 Forward propagation in neural network

In [7]:
# function for applying forward propagation in L layer network

def L_layer_forward_prop(X, parameters):
    """
    Arguments are,
    X ---> Input data (numpy array)
    parameters ---> A python dictionary containing the parameters of the network. i.e., Weights and bias in each layer
                    Output of the function - initialize_parameters()
                    
    Function returns are,
    A_L_layer ---> Activation from the output layer (L-th layer)
    backup_values ---> for storing values of forward_activation()
    
    """
    
    A = X
    backup_values_tot = []
    L = len(parameters) // 2
    
    for l in range(1, L):
        A_prev_layer = A
        
        A_L_layer, backup_values = forward_activation(A, parameters["W" + str(l)], parameters["b" + str(l)], activation = "relu")
        backup_values_tot.append(backup_values)
        
    return A_L_layer, backup_values_tot

#### 4. Cost function

In [9]:
# function for calculating cost - J

def cost(A_L_layer, Y):
    """
    Arguments are,
    A_L_layer ---> Vector corresponding to predictions of label with shape (1, number of examples)
    Y ---> Label vector of shape (1, number of examples)
    
    Function returns are,
    Cost ---> Cost of the predictions
    
    """
    m = Y.shape[1]
    cost = -1/m * (np.dot(np.log(A_L_layer),Y.T) + np.dot(np.log(1-A_L_layer),(1-Y).T))
    cost = np.squeeze(cost)
                   
    return cost

#### 5. Backward propagation in neural network

#### 5.1 Linear backward function

In [1]:
# function for calculating linear backward

def linear_backward(dZ, backup_values):
    """
    Arguments are,
    dZ ---> Gradient of the cost with respect to the linear output of each layer - dJ/dZ(l)
    backup_values ---> Values of A_prev_layer, W and b from the forward propagation step - tuple
    
    Function returns are,
    dW ---> Gradient of cost with respect to W
    db ---> Gradient of cost with respect to b
    dA_prev_layer ---> Gradient of cost with respect to the activation from the previous layer
    
    """
    A_prev_layer, W, b = backup_values
    m = A_prev_layer.shape[1]
    
    dW = 1/m * np.dot(dZ, A_prev_layer.T)
    db = 1/m * np.sum(dZ, axis = 1, keepdims = True)
    dA_prev_layer = np.dot(W.T, dZ)
    
    return dA_prev_layer, dW, db

#### 5.2 Linear backward activtion function

In [2]:
# functon for backward step for activation

def backward_activation(dA, backup_values, activation = "relu"):
    """
    Arguments are,
    dA ---> Activation gradient of the current layer
    backup_values ---> Stored values of forward activation and linear step
    activation ---> Activation to be used (ReLu) i.e., A = RELU(Z) = max(0,Z)
    
    Function returns are,
    dA_prev_layer ---> Gradient of cost with respect to the activation of the previous layer
    dW ---> Gradient of the cost with respect to W
    db ---> Gradient of the cost with respect to b
    
    """
    (linear_backup_values, activation_backup_values) = backup_values
    
    dZ = dA * np.max(0,Z)
    dA_prev_layer, dW, db = linear_backward(dZ, linear_backup_values)
    
    return dA_prev_layer, dW, db

#### 5.3 Backward propagation in neural network

In [4]:
# function for applying backward propagation in L layer network

def L_layer_backward_prop(A_L_layer, Y, backup_values_tot):
    """
    Arguments are,
    A_L_layer ---> Vector corresponding to predictions - Output of forward propagation
    Y ---> Label vector of shape (1, number of examples)
    backup_values_tot ---> All backup_values of forward activation
    
    Function returns are,
    gradients ---> Dictionary containing gradients i.e., dA, dW and db
    
    """
    gradients = {}
    m = A_L_layer.shape[1]
    Y = Y.reshape(A_L_layer.shape)
    L = len(backup_values_tot)
    
    # initializing backpropagation
    
    dA_L_layer = -(np.divide(Y, A_L_layer) - np.divide(1-Y, 1-A_L_layer))
    
    # looping from l-1 to L=0
    for l in reversed(range(L)):
        current_value = backup_values_tot[l-1]
        dA_prev_layer_tmp, dW_tmp, db_tmp = backward_activation(dA_prev_layer_tmp, current_value, activation = "relu")
        gradients["dA" + str(l-1)] = dA_prev_layer_tmp
        gradients["dW" + str(l)] = dW_tmp
        gradients["db" + str(l)] = db_tmp
        
    return gradients

#### 6. Parameter updation

In [5]:
# function for updating parameters

def update_params(params, gradients, learning_rate):
    """
    Arguments are,
    params ---> Dictionary of parameters
    gradients ---> Dictionary of gradients
    learning_rate ---> Determines the step size of each iterations
    
    Function returns are,
    parametrs ---> Dictionary containing the updated parameters
    
    """
    
    parameters = params.copy()
    L = len(parameters) // 2 
    
    for l in range(L):
        
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * gradients["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * gradients["db" + str(l+1)]
        
    return parameters